# 🏗️ Digital Construction Archive (DCA) - CSV zu RDF Pipeline für RenkuLab

**Zweck**: Migration der DROID CSV-Daten zu RDF basierend auf DCA-Ontologie  
**Zielumgebung**: RenkuLab Jupyter Notebook  
**Standards**: RiC-O, PREMIS, Dublin Core mit DCA-spezifischen Erweiterungen  
**Datum**: Februar 2026  

---

## ⚡ Quick Start Guide
1. **Kopieren Sie jede Zelle einzeln** zu RenkuLab  
2. **Führen Sie sie der Reihe nach aus**  
3. **Passen Sie die Pfade** in Schritt 2 an Ihre RenkuLab-Struktur an  
4. **Überwachen Sie Output** für Fehler oder Warnungen  

## 📦 Schritt 1: RenkuLab Dependencies & Core Imports

In [ ]:
# =====================================================
# FIRST RUN IN RENKULAB: Install dependencies
# =====================================================
# !pip install rdflib pandas exifread pillow networkx

from pathlib import Path
import pandas as pd
import json, subprocess, hashlib, sys, math, re
from datetime import datetime
from typing import Optional, Dict, List, Set, Union
import warnings
warnings.filterwarnings('ignore')

# RDF Core Libraries
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS
from rdflib.plugins.serializers.turtle import TurtleSerializer

# Optional: Network analysis for provenance graphs
try:
    import networkx as nx
    NX_AVAILABLE = True
except ImportError:
    print("⚠️  NetworkX not available - provenance graphs disabled")
    NX_AVAILABLE = False

print("✅ All dependencies loaded successfully")
print(f"📅 DCA Pipeline started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Python {sys.version_info.major}.{sys.version_info.minor}")
print(f"📚 RDFLib version: {getattr(Graph(), 'version', 'unknown')}")

## 🏗️ Schritt 2: Digital Construction Archive (DCA) Ontology Setup

**Wichtig**: Diese Namespaces entsprechen exakt dem ETH DCA Standard

In [ ]:
# =====================================================
# DCA ONTOLOGY NAMESPACES - ETH Zürich Standard
# =====================================================

# Core DCA Namespaces
DCA      = Namespace("http://dca.ethz.ch/ontology#")     # DCA Classes & Properties
DCA_ID   = Namespace("http://dca.ethz.ch/id/")          # Individual Resources
DCA_TECH = Namespace("http://dca.ethz.ch/tech#")        # Technical Properties

# International Standards
PREMIS   = Namespace("http://www.loc.gov/premis/rdf/v3/")  # Digital Preservation Metadata
RICO     = Namespace("https://www.ica.org/standards/RiC/ontology#")  # Records in Contexts
# DCTERMS already imported from rdflib.namespace

# Utility
OWL      = Namespace("http://www.w3.org/2002/07/owl#")
XSDNS    = XSD  # Shorthand

# =====================================================
# DCA FILE TYPE DEFINITIONS
# =====================================================

# Image file extensions (construction documentation)
IMG_EXTENSIONS = {
    "jpg", "jpeg", "tif", "tiff", "png", "gif", "bmp", "webp",
    # RAW formats (professional photography)
    "dng", "cr2", "cr3", "nef", "arw", "orf", "rw2"
}

# Adobe Creative Suite formats
ADOBE_EXTENSIONS = {
    "psd", "psb", "ai", "indd", "idml", "eps",
    "pdf"  # Often contains rich Adobe metadata
}

# CAD & Engineering formats
CAD_EXTENSIONS = {
    "dwg", "dxf", "step", "stp", "iges", "igs", 
    "ifc", "3dm", "skp"  # BIM/Architecture specific
}

# Combined target extensions for metadata enrichment
TARGET_EXTENSIONS = IMG_EXTENSIONS | ADOBE_EXTENSIONS | CAD_EXTENSIONS

print(f"🏗️  DCA Ontology initialized")
print(f"📁 Target file types: {len(TARGET_EXTENSIONS)} extensions")
print(f"📸 Image types: {len(IMG_EXTENSIONS)}")
print(f"🎨 Adobe types: {len(ADOBE_EXTENSIONS)}")
print(f"📐 CAD types: {len(CAD_EXTENSIONS)}")

## 🔗 Schritt 3: DCA ID Generation - MD5-Based Consistent Identifiers

**Kritisch**: Diese Funktionen müssen identische IDs wie das ETZ DCA System erzeugen!

In [ ]:
# =====================================================
# DCA ID GENERATION - MD5-BASED SYSTEM
# =====================================================

# Base URI for all DCA identifiers
DCA_ID_BASE = "http://dca.ethz.ch/id/"

def dca_file_uri_from_md5(md5_hex: Optional[str]) -> Optional[URIRef]:
    """
    Creates dca-id:file_<md5[:16]> from MD5 hex string (DROID format).
    
    Args:
        md5_hex: MD5 hash as hex string (32 chars)
        
    Returns:
        URIRef or None if invalid input
        
    Example:
        md5 = "a1b2c3d4e5f6789012345678901234ab"
        → dca-id:file_a1b2c3d4e5f67890
    """
    if not md5_hex or not isinstance(md5_hex, str):
        return None
    
    # Clean and validate
    md5_clean = md5_hex.strip().lower()
    if len(md5_clean) < 16 or not re.match(r'^[a-f0-9]+$', md5_clean):
        return None
    
    # Use first 16 characters for consistent short IDs
    short_id = md5_clean[:16]
    return URIRef(DCA_ID_BASE + f"file_{short_id}")

def dca_file_uri_from_path_fallback(file_path: str) -> URIRef:
    """
    Fallback ID generation from file path (when MD5 not available).
    
    WARNING: Only use this if DROID MD5 is unavailable!
    Path-based IDs are less stable than content-based MD5.
    """
    path_normalized = str(Path(file_path)).replace('\\', '/')  # Normalize separators
    path_hash = hashlib.sha256(path_normalized.encode('utf-8')).hexdigest()[:16]
    return URIRef(DCA_ID_BASE + f"file_{path_hash}")

def dca_project_uri(project_name: str) -> URIRef:
    """
    Generate project URI following DCA conventions.
    
    Example: "WeingutGantenbein" → dca-id:project_WeingutGantenbein
    """
    # Sanitize project name for URI use
    sanitized = re.sub(r'[^a-zA-Z0-9_-]', '_', project_name)
    return URIRef(DCA_ID_BASE + f"project_{sanitized}")

def dca_activity_uri(activity_name: str) -> URIRef:
    """
    Generate activity URI for provenance tracking.
    
    Example: "ArchivingGantenbein2026" → dca-id:ArchivingGantenbein2026
    """
    sanitized = re.sub(r'[^a-zA-Z0-9_-]', '_', activity_name)
    return URIRef(DCA_ID_BASE + sanitized)

# Test the ID generation
test_md5 = "a1b2c3d4e5f6789012345678901234ab"
test_uri = dca_file_uri_from_md5(test_md5)
print(f"✅ ID Generation Functions Ready")
print(f"🧪 Test MD5: {test_md5}")
print(f"🆔 Generated URI: {test_uri}")
print(f"📏 ID Length: {len(str(test_uri).split('_')[1])} characters")

## 📂 Schritt 4: RenkuLab Pfad-Konfiguration

**ANPASSEN**: Diese Pfade müssen an Ihre RenkuLab-Projektstruktur angepasst werden!

In [ ]:
# =====================================================
# RENKULAB PATH CONFIGURATION
# =====================================================
# TODO: Adjust these paths to match your RenkuLab project structure!

# Input: DROID CSV results
DROID_CSV_PATH = Path("/home/renku/work/data/gramazio-kohler-archiv-server_DROIDresults.csv")
# Alternative if in project subdirectory:
# DROID_CSV_PATH = Path("./data/droid_results.csv")

# Input: Base directory containing the actual files (for ExifTool)
FILES_BASE_DIR = Path("/home/renku/work/data/gramazio-kohler-archiv-server/")
# Alternative:
# FILES_BASE_DIR = Path("./data/files/")

# Output: Generated RDF file
OUTPUT_RDF_PATH = Path("./dca_catalog_generated.ttl")

# Optional: Existing RDF to merge with
EXISTING_RDF_PATH = Path("./existing_catalog.ttl")  # Set to None if not available

# ExifTool configuration
EXIFTOOL_COMMAND = "exiftool"  # Adjust if installed in different location
# For custom installation: 
# EXIFTOOL_COMMAND = "/home/renku/work/tools/exiftool/exiftool"

# Project metadata
PROJECT_NAME = "WeingutGantenbein"  # Will become dca-id:project_WeingutGantenbein
ACTIVITY_NAME = "ArchivingGantenbein2026"  # Provenance activity

# Validation
print("📂 RenkuLab Path Configuration:")
print(f"📊 DROID CSV: {DROID_CSV_PATH}")
print(f"📁 Files Base: {FILES_BASE_DIR}")
print(f"💾 Output RDF: {OUTPUT_RDF_PATH}")
print(f"🏗️  Project: {PROJECT_NAME}")

# Check critical paths
if DROID_CSV_PATH.exists():
    print(f"✅ DROID CSV found: {DROID_CSV_PATH.stat().st_size / 1024 / 1024:.1f} MB")
else:
    print(f"❌ DROID CSV not found: {DROID_CSV_PATH}")
    print("   → Please update DROID_CSV_PATH to correct location")

if FILES_BASE_DIR.exists():
    print(f"✅ Files base directory found")
else:
    print(f"❌ Files base directory not found: {FILES_BASE_DIR}")
    print("   → ExifTool operations will be skipped")

## 📊 Schritt 5: DROID CSV Laden & Analysieren

In [ ]:
# =====================================================
# LOAD AND ANALYZE DROID CSV DATA
# =====================================================

def load_droid_csv(csv_path: Path) -> pd.DataFrame:
    """
    Load DROID CSV with proper encoding and error handling.
    
    Returns DataFrame with standardized column names.
    """
    try:
        # Try UTF-8 first
        df = pd.read_csv(csv_path, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            # Fallback to latin-1
            df = pd.read_csv(csv_path, encoding='latin-1')
            print("⚠️  Used latin-1 encoding for CSV")
        except Exception as e:
            print(f"❌ Failed to read CSV: {e}")
            return pd.DataFrame()
    
    print(f"📊 DROID CSV loaded: {len(df):,} records")
    print(f"📋 Columns: {list(df.columns)}")
    
    return df

def analyze_droid_data(df: pd.DataFrame) -> Dict[str, any]:
    """
    Analyze DROID data for DCA processing insights.
    """
    if df.empty:
        return {}
    
    analysis = {}
    
    # File type analysis
    if 'EXT' in df.columns:
        ext_counts = df['EXT'].value_counts()
        analysis['extensions'] = ext_counts.head(20).to_dict()
        
        # Target extensions analysis
        df['ext_lower'] = df['EXT'].str.lower() if 'EXT' in df.columns else ''
        target_mask = df['ext_lower'].isin(TARGET_EXTENSIONS)
        analysis['target_files'] = target_mask.sum()
        analysis['total_files'] = len(df)
    
    # MD5 hash analysis
    md5_col = None
    for col in ['MD5_HASH', 'HASH', 'md5', 'MD5']:
        if col in df.columns:
            md5_col = col
            break
    
    if md5_col:
        analysis['md5_column'] = md5_col
        analysis['md5_available'] = df[md5_col].notna().sum()
    else:
        analysis['md5_column'] = None
        analysis['md5_available'] = 0
    
    return analysis

# Load the data
print("🔄 Loading DROID CSV data...")
droid_df = load_droid_csv(DROID_CSV_PATH)

if not droid_df.empty:
    analysis = analyze_droid_data(droid_df)
    
    print("\n📈 DROID Data Analysis:")
    print(f"📊 Total files: {analysis.get('total_files', 0):,}")
    print(f"🎯 Target files (img/adobe/cad): {analysis.get('target_files', 0):,}")
    print(f"#️⃣  MD5 hashes available: {analysis.get('md5_available', 0):,}")
    if analysis.get('md5_column'):
        print(f"🔑 MD5 column: '{analysis['md5_column']}'")
    
    if analysis.get('extensions'):
        print("\n📁 Top file extensions:")
        for ext, count in list(analysis['extensions'].items())[:10]:
            print(f"   .{ext}: {count:,} files")
else:
    print("❌ No DROID data available - check file path!")

## 🏗️ Schritt 6: RDF Graph Initialisierung mit DCA Ontology

In [ ]:
# =====================================================
# INITIALIZE RDF GRAPH WITH DCA ONTOLOGY
# =====================================================

def create_dca_graph() -> Graph:
    """
    Create RDF graph with DCA ontology structure and namespace bindings.
    """
    g = Graph()
    
    # Bind namespaces for clean Turtle output
    g.bind("dca", DCA)
    g.bind("dca-id", DCA_ID)
    g.bind("dca-tech", DCA_TECH)
    g.bind("premis", PREMIS)
    g.bind("rico", RICO)
    g.bind("dcterms", DCTERMS)
    g.bind("owl", OWL)
    g.bind("xsd", XSD)
    
    return g

def add_ontology_definitions(g: Graph):
    """
    Add DCA ontology class and property definitions.
    """
    # Ontology declaration
    ontology_uri = URIRef("http://dca.ethz.ch/ontology")
    g.add((ontology_uri, RDF.type, OWL.Ontology))
    g.add((ontology_uri, DCTERMS.created, Literal(datetime.now().strftime("%Y-%m-%d"), datatype=XSD.date)))
    g.add((ontology_uri, DCTERMS.creator, Literal("ETH Zurich - Digital Construction Archive Project")))
    g.add((ontology_uri, DCTERMS.description, 
           Literal("Standards-based ontology for digital construction archives using RiC-O, PREMIS, and Dublin Core with clean namespace separation and Nextcloud WebDAV integration", lang="en")))
    
    # DCA Classes
    g.add((DCA.ConstructionProject, RDF.type, OWL.Class))
    g.add((DCA.ConstructionProject, RDFS.label, Literal("Construction Project", lang="en")))
    g.add((DCA.ConstructionProject, RDFS.comment, 
           Literal("A construction or architectural project containing digital files", lang="en")))
    g.add((DCA.ConstructionProject, RDFS.subClassOf, RICO.RecordSet))
    
    g.add((DCA.ArchiveFile, RDF.type, OWL.Class))
    g.add((DCA.ArchiveFile, RDFS.label, Literal("Archive File", lang="en")))
    g.add((DCA.ArchiveFile, RDFS.comment, 
           Literal("A digital file within the construction archive", lang="en")))
    g.add((DCA.ArchiveFile, RDFS.subClassOf, PREMIS.Object))
    g.add((DCA.ArchiveFile, RDFS.subClassOf, RICO.Record))
    
    # DCA Properties
    g.add((DCA.belongsToProject, RDF.type, OWL.ObjectProperty))
    g.add((DCA.belongsToProject, RDFS.label, Literal("belongs to project", lang="en")))
    g.add((DCA.belongsToProject, RDFS.comment, 
           Literal("Indicates that a file belongs to a specific construction project", lang="en")))
    g.add((DCA.belongsToProject, RDFS.domain, DCA.ArchiveFile))
    g.add((DCA.belongsToProject, RDFS.range, DCA.ConstructionProject))
    
    # Technical properties
    g.add((DCA_TECH.sourcePath, RDF.type, OWL.DatatypeProperty))
    g.add((DCA_TECH.sourcePath, RDFS.label, Literal("source path", lang="en")))
    g.add((DCA_TECH.sourcePath, RDFS.comment, 
           Literal("DEPRECATED: Original filesystem path from DROID analysis. Use dcterms:identifier with Nextcloud WebDAV URL instead to avoid redundancy.", lang="en")))
    g.add((DCA_TECH.sourcePath, RDFS.domain, RICO.Record))
    g.add((DCA_TECH.sourcePath, RDFS.range, XSD.string))
    g.add((DCA_TECH.sourcePath, OWL.deprecated, Literal(True)))

def add_project_and_activity(g: Graph, project_name: str, activity_name: str):
    """
    Add project and provenance activity to graph.
    """
    # Project
    project_uri = dca_project_uri(project_name)
    g.add((project_uri, RDF.type, DCA.ConstructionProject))
    g.add((project_uri, RDF.type, RICO.RecordSet))
    g.add((project_uri, DCTERMS.title, Literal(project_name)))
    
    # Activity
    activity_uri = dca_activity_uri(activity_name)
    g.add((activity_uri, RDF.type, RICO.Activity))
    g.add((activity_uri, RDFS.label, Literal(f"Digitale Archivierung {project_name} Konstruktionsunterlagen")))
    g.add((activity_uri, DCTERMS.description, 
           Literal(f"Systematische digitale Erfassung und Archivierung der Konstruktionsdokumentation des {project_name} Projekts. Enthält Pläne, Fotos, technische Berichte und Korrespondenz.")))
    g.add((activity_uri, RICO.hasEndDate, Literal("Februar 2026")))
    g.add((activity_uri, RICO.occurredAtDate, Literal(datetime.now().isoformat(), datatype=XSD.dateTime)))
    g.add((activity_uri, RICO.technique, 
           Literal("DROID file identification with PRONOM registry, RDF metadata generation, Nextcloud WebDAV storage")))
    
    # Team
    team_uri = URIRef(DCA_ID_BASE + "DCA_Team")
    g.add((team_uri, RDF.type, RICO.Group))
    g.add((team_uri, RDFS.label, Literal("Digital Construction Archive Team, ETH Zürich")))
    g.add((activity_uri, RICO.isOrWasPerformedBy, team_uri))
    
    # Link project to activity
    g.add((project_uri, RICO.isOrWasDocumentedBy, activity_uri))
    
    return project_uri, activity_uri

# Initialize the graph
print("🔄 Initializing DCA RDF Graph...")
graph = create_dca_graph()

# Add ontology structure
add_ontology_definitions(graph)
print("✅ DCA ontology definitions added")

# Add project and activity
project_uri, activity_uri = add_project_and_activity(graph, PROJECT_NAME, ACTIVITY_NAME)
print(f"✅ Project added: {project_uri}")
print(f"✅ Activity added: {activity_uri}")

print(f"📊 Current graph size: {len(graph)} triples")

## 🔄 Schritt 7: DROID Daten zu RDF Konvertierung

In [ ]:
# =====================================================
# CONVERT DROID DATA TO RDF
# =====================================================

def safe_literal(value, datatype=None, lang=None):
    """
    Create RDF literal with error handling.
    """
    if pd.isna(value) or value == '':
        return None
    try:
        return Literal(str(value), datatype=datatype, lang=lang)
    except Exception as e:
        print(f"⚠️  Literal creation failed for '{value}': {e}")
        return Literal(str(value))  # Fallback without datatype

def add_premis_identifier(g: Graph, file_uri: URIRef, id_type: str, value: str):
    """
    Add PREMIS identifier as blank node.
    """
    if not value or pd.isna(value):
        return
    
    bn = BNode()
    g.add((file_uri, PREMIS.hasIdentifier, bn))
    g.add((bn, PREMIS.identifierType, Literal(id_type)))
    g.add((bn, PREMIS.identifierValue, Literal(str(value))))

def process_droid_record(g: Graph, record: pd.Series, project_uri: URIRef) -> Optional[URIRef]:
    """
    Process single DROID record and add to RDF graph.
    
    Returns the created file URI or None if processing failed.
    """
    try:
        # Determine MD5 hash column (DROID CSV variations)
        md5_hash = None
        for col in ['MD5_HASH', 'HASH', 'md5', 'MD5']:
            if col in record.index and not pd.isna(record.get(col)):
                md5_hash = record[col]
                break
        
        # Generate file URI
        if md5_hash:
            file_uri = dca_file_uri_from_md5(md5_hash)
        else:
            # Fallback to path-based ID
            file_path = record.get('FILE_PATH', record.get('PATH', ''))
            if not file_path:
                return None
            file_uri = dca_file_uri_from_path_fallback(file_path)
        
        if not file_uri:
            return None
        
        # Core classes
        g.add((file_uri, RDF.type, DCA.ArchiveFile))
        g.add((file_uri, RDF.type, PREMIS.Object))
        g.add((file_uri, RDF.type, RICO.Record))
        
        # Project relationship
        g.add((file_uri, DCA.belongsToProject, project_uri))
        g.add((file_uri, RICO.isOrWasIncludedIn, project_uri))
        
        # Basic metadata
        if 'NAME' in record.index:
            g.add((file_uri, DCTERMS.title, safe_literal(record['NAME'])))
        
        # File path as identifier (WebDAV-style if possible)
        file_path = record.get('FILE_PATH', record.get('PATH', ''))
        if file_path:
            # Convert to WebDAV URL format if possible
            webdav_url = f"https://nextcloud.ethz.ch/remote.php/dav/files/padrian/DCA/{PROJECT_NAME}/{file_path}"
            g.add((file_uri, DCTERMS.identifier, URIRef(webdav_url)))
        
        # Timestamps
        for time_col in ['LAST_MODIFIED', 'MODIFIED', 'DATE_MODIFIED']:
            if time_col in record.index and not pd.isna(record[time_col]):
                timestamp = safe_literal(record[time_col], datatype=XSD.dateTime)
                if timestamp:
                    g.add((file_uri, DCTERMS.modified, timestamp))
                break
        
        # PREMIS format information
        if 'FORMAT_NAME' in record.index:
            g.add((file_uri, PREMIS.hasFormatName, safe_literal(record['FORMAT_NAME'])))
        
        # Format details
        format_notes = []
        if 'MIME_TYPE' in record.index and not pd.isna(record['MIME_TYPE']):
            format_notes.append(f"MIME: {record['MIME_TYPE']}")
        if 'PUID' in record.index and not pd.isna(record['PUID']):
            format_notes.append(f"PRONOM ID: {record['PUID']}")
        
        for note in format_notes:
            g.add((file_uri, PREMIS.hasFormatNote, Literal(note)))
        
        # File size
        if 'SIZE' in record.index and not pd.isna(record['SIZE']):
            try:
                size_val = int(float(record['SIZE']))
                g.add((file_uri, PREMIS.hasSize, Literal(size_val, datatype=XSD.long)))
            except (ValueError, TypeError):
                pass
        
        # DROID identification method
        g.add((file_uri, PREMIS.hasCreatingApplication, Literal("DROID: Signature")))
        
        return file_uri
        
    except Exception as e:
        print(f"⚠️  Failed to process record: {e}")
        return None

# Process DROID data
if not droid_df.empty:
    print(f"🔄 Processing {len(droid_df):,} DROID records...")
    
    processed_count = 0
    error_count = 0
    
    for idx, record in droid_df.iterrows():
        file_uri = process_droid_record(graph, record, project_uri)
        if file_uri:
            processed_count += 1
        else:
            error_count += 1
        
        # Progress indicator
        if (idx + 1) % 1000 == 0:
            print(f"   Processed: {idx + 1:,} / {len(droid_df):,} records")
    
    print(f"✅ DROID processing complete:")
    print(f"   📊 Successfully processed: {processed_count:,} files")
    print(f"   ❌ Errors: {error_count:,} files")
    print(f"   📈 Graph size: {len(graph):,} triples")
else:
    print("❌ No DROID data to process")

## 💾 Schritt 8: RDF Export & Validierung

In [ ]:
# =====================================================
# EXPORT RDF AND VALIDATE
# =====================================================

def export_rdf_turtle(g: Graph, output_path: Path) -> bool:
    """
    Export RDF graph to Turtle format with proper formatting.
    """
    try:
        # Serialize to Turtle with nice formatting
        turtle_data = g.serialize(format='turtle')
        
        # Write to file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(turtle_data)
        
        file_size = output_path.stat().st_size
        print(f"✅ RDF exported to: {output_path}")
        print(f"📊 File size: {file_size / 1024 / 1024:.2f} MB")
        print(f"📈 Triples: {len(g):,}")
        
        return True
        
    except Exception as e:
        print(f"❌ Export failed: {e}")
        return False

def validate_rdf_content(g: Graph) -> Dict[str, int]:
    """
    Validate RDF content and provide statistics.
    """
    stats = {}
    
    # Count by type
    stats['archive_files'] = len(list(g.subjects(RDF.type, DCA.ArchiveFile)))
    stats['projects'] = len(list(g.subjects(RDF.type, DCA.ConstructionProject)))
    stats['activities'] = len(list(g.subjects(RDF.type, RICO.Activity)))
    
    # Count properties
    stats['has_title'] = len(list(g.subjects(DCTERMS.title, None)))
    stats['has_identifier'] = len(list(g.subjects(DCTERMS.identifier, None)))
    stats['has_format'] = len(list(g.subjects(PREMIS.hasFormatName, None)))
    stats['has_size'] = len(list(g.subjects(PREMIS.hasSize, None)))
    
    return stats

# Export RDF
print("💾 Exporting RDF to Turtle format...")
export_success = export_rdf_turtle(graph, OUTPUT_RDF_PATH)

# Validate content
if export_success:
    print("\n🔍 RDF Content Validation:")
    stats = validate_rdf_content(graph)
    
    for key, value in stats.items():
        print(f"   {key.replace('_', ' ').title()}: {value:,}")
    
    # Quality checks
    if stats.get('archive_files', 0) > 0:
        coverage_title = (stats.get('has_title', 0) / stats['archive_files']) * 100
        coverage_format = (stats.get('has_format', 0) / stats['archive_files']) * 100
        
        print(f"\n📊 Metadata Coverage:")
        print(f"   Titles: {coverage_title:.1f}%")
        print(f"   Format info: {coverage_format:.1f}%")
    
    print(f"\n✅ RDF generation complete!")
    print(f"📂 Output file: {OUTPUT_RDF_PATH.absolute()}")
else:
    print("❌ RDF export failed")

## 🎯 Schritt 9: Beispiel-Abfragen zur Verifikation

Diese Queries helfen dabei, die generierte RDF zu validieren.

In [ ]:
# =====================================================
# SAMPLE SPARQL QUERIES FOR VALIDATION
# =====================================================

def run_sample_queries(g: Graph):
    """
    Run sample SPARQL queries to validate RDF structure.
    """
    
    queries = {
        "Total Files": """
            SELECT (COUNT(?file) AS ?count) WHERE {
                ?file a dca:ArchiveFile .
            }
        """,
        
        "Image Files": """
            SELECT ?file ?title ?format WHERE {
                ?file a dca:ArchiveFile ;
                      dcterms:title ?title ;
                      premis:hasFormatName ?format .
                FILTER(CONTAINS(LCASE(?format), "image") || 
                       CONTAINS(LCASE(?format), "jpeg") || 
                       CONTAINS(LCASE(?format), "tiff"))
            } LIMIT 10
        """,
        
        "Project Info": """
            SELECT ?project ?title ?fileCount WHERE {
                ?project a dca:ConstructionProject ;
                         dcterms:title ?title .
                {
                    SELECT ?project (COUNT(?file) AS ?fileCount) WHERE {
                        ?file dca:belongsToProject ?project .
                    } GROUP BY ?project
                }
            }
        """,
        
        "Format Distribution": """
            SELECT ?format (COUNT(?file) AS ?count) WHERE {
                ?file a dca:ArchiveFile ;
                      premis:hasFormatName ?format .
            } GROUP BY ?format ORDER BY DESC(?count) LIMIT 10
        """
    }
    
    print("🔍 Running validation queries...\n")
    
    for query_name, query_text in queries.items():
        try:
            print(f"📊 {query_name}:")
            results = g.query(query_text)
            
            for row in results:
                values = [str(val) for val in row]
                print(f"   {' | '.join(values)}")
            
            print(f"   ({len(results)} results)\n")
            
        except Exception as e:
            print(f"   ❌ Query failed: {e}\n")

# Run validation queries
if len(graph) > 0:
    run_sample_queries(graph)
else:
    print("⚠️  No data in graph to query")

## ✅ Pipeline Abschluss & Nächste Schritte

Die CSV-zu-RDF Pipeline ist abgeschlossen. Die generierte RDF-Datei kann nun:

### 📤 Zurück nach ETH DCA übertragen werden
- Upload der `.ttl` Datei
- Merge mit bestehenden RDF-Daten
- Integration in Triple Store (Apache Jena Fuseki)

### 🔄 Für weitere Verarbeitung verwendet werden
- XMP-Metadaten Extraktion mit ExifTool
- Provenience-Daten aus Adobe Creative Suite
- Verlinkung zwischen verwandten Dateien

### 📊 Für Analysen und Visualisierungen
- SPARQL-Queries für Statistiken
- Netzwerk-Analysen der Datei-Dependencies
- Timeline-Visualisierungen der Projektentwicklung

In [ ]:
# =====================================================
# PIPELINE SUMMARY
# =====================================================

end_time = datetime.now()
print(f"🏁 DCA CSV-zu-RDF Pipeline abgeschlossen")
print(f"📅 Ende: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Finale Graph-Größe: {len(graph):,} triples")
print(f"📄 Output-Datei: {OUTPUT_RDF_PATH.absolute()}")

if OUTPUT_RDF_PATH.exists():
    file_size_mb = OUTPUT_RDF_PATH.stat().st_size / 1024 / 1024
    print(f"📏 Dateigröße: {file_size_mb:.2f} MB")
    print(f"✅ Bereit für Transfer zu ETH DCA System")
else:
    print(f"❌ Output-Datei wurde nicht erstellt")

print("\n🔄 Nächste Schritte:")
print("   1. RDF-Datei nach ETH DCA System übertragen")
print("   2. Mit bestehenden Daten mergen")
print("   3. ExifTool für XMP-Metadaten einsetzen")
print("   4. SPARQL-Endpoint für Abfragen einrichten")